# Knuth-Morris-Pratt

LPS array is the key.  
let the string be like this.  
**abcaba + qwertyuytrew + abcabc**
in the last string till last b our length will be increasing, because characters matched till that point.  
but at **c** it will not match, but we know that before **c** our string matches so we will go 1 length less and try to match it again and keep doing it untill it becomes 0.  
```
abcaba 
abcabc
123456 

l = 6, a != c, so l-=1
l = 5, b != c, so l-=1
l = 4, a != c, so l-=1
l = 3, c == c, so ans = l = 3.
```

In [7]:
# wiki and geeks for geeks are enough to recollect.
def longest_prefix(patter):
    n = len(patter)
    index, length = 1, 0 
    longest_prefix_sufix = [0]*n
    while index<n: 
        if patter[index] == patter[length]:
            length += 1
            longest_prefix_sufix[index] = length
            index += 1
        else:
            if length:
                length = longest_prefix_sufix[length-1]
            else:
                longest_prefix_sufix[index] = 0
                index += 1
    return longest_prefix_sufix

def KMPSearch(pat, string):
    n, m = len(string), len(pat)
    lps = longest_prefix(pat)
    print(lps)
    i = 0
    j = 0
    matched = []
    while i<n:
        # If pattern Matched
        if pat[j] == string[i]:
            i+=1
            j+=1
            
        # If all of them matched.
        if j == m:
            matched.append(i+1-m)
            j = lps[j-1]
        # if not matched.
        elif i<n and pat[j] != string[i]:
            # if j not zero go back to previous matched string.
            if j != 0:
                j = lps[j-1]
            # if j is zero just compare it to next letter.
            else:
                i += 1
    return matched
        
string = "abcdabcdabceabcd"
pat = "abcbwabcbq"

print(KMPSearch(pat, string))

[0, 0, 0, 0, 0, 1, 2, 3, 4, 0]
[]


# Rabin-Karp method
This is more like efficient hashing and with mo's sliding window technique.  
`NOTE: can be used in Mo's algorithm as successive operations are O(1)`

In [7]:
# code
s = 'abcdefghijklmnopqrstuvwxyz'
p = 'lmno'
A = lambda x: ord(x) - ord('a') + 1
# mapping tools
m = 31
mod = int(1e9)+7
a, b = len(s), len(p)
# memoization of powers of 'm' to not calculate again.
power = [pow(m, i, mod) for i in range(max(a, b))]

# ph hash value of 'p', sh hash value of 's'
ph = 0; sh = 0

# calculating intitial values of p and s strings
for i in range(b-1, -1, -1):
    ph = (ph + A(p[i])*power[b-i-1])%mod
    sh = (sh + A(s[i])*power[b-i-1])%mod
ans = 0
if ph == sh: print(f"found at {0}")

# O(1) successive calculation of has value for substirng of 's'
for i in range(b, a):
    sh = ((sh - A(s[i-b])*power[b-1])*m + A(s[i]))%mod
    if sh == ph: ans += 1; print(f"found at {i-b+1}")
print(f"total found {ans}")    

found at 11
total found 1


In [3]:
# ez way to dp rabin karp, go in reverse order because modulo makes it hard to divide.

def RabinKarp(string, pattern):
    d = {chr(a): i for i, a in enumerate(range(0, 184), 1)}
    def hash_(s, power, mod): return sum(d[v]*pow(power, i, mod) for i, v in enumerate(s))%mod
    
    p, mod = 31, 10**9 + 7
    n, m = len(string), len(pattern)
    ph = hash_(pattern, p, mod)
    sh = 0
    pm = pow(p, m, mod)

    found = 0
    for i in reversed(range(n)):
        sh = (sh*p + d[string[i]])%mod
        if i+m<n:
            sh = (sh - d[string[i+m]]*pm)%mod
        if sh == ph:
            print(f"found at index: {i}")    
            found += 1
    print(f"Total found: {found}")

string = 'abcdefghijklmnopqrstuvwxyz'
pattern = 'lmno'
RabinKarp(string, pattern)

found at index: 11
Total found: 1


# Z Function

Let *s* is our string. **z function** returns a list of length of string, where list\[i\] represents length of prefix which is common in actual string and sufix of that string starting from ith index.

## Brute Force 

In [34]:
s = 'aaaabaa'

# simple
def zfuncBruteSimple(s):
    n = len(s)
    lis = [0]*n
    
    for i in range(1, n):
        k = 0
        while i+k<n and s[k] == s[k+i]: k += 1
        lis[i] = k
    return lis

# rather than making a variable k to store the lenght while the condition is valid, 
# we can directly use lis[i] since it is same vairable and is default 0
def zfuncBruteBetter(s):
    n = len(s)
    lis = [0]*n
    for i in range(1, n):
        while i+lis[i]<n and s[lis[i]] == s[lis[i]+i]: lis[i] += 1
    return lis

print(zfuncBruteBetter(s))

[0, 3, 2, 1, 0, 2, 1]


Make a range \[l, r\], to tell that we know ans till r.  
if we are outside this range, then use burte force technique.  
lets say we got answer 4 at index i after brute force.  
then our l is i and our r = i+4, so at jth iteration, rather than starting from start we can start from z\[j-l\].

In [3]:
s = "abcabcabca"

def zfuncOpt(s):
    n = len(s)
    z = [0]*n
    l = r = 0
    for i in range(1, n):
        if i<=r:
            z[i] = min(r-i+1, z[i-l])
        while i+z[i]<n and s[z[i]] == s[i+z[i]]: z[i]+=1
        if i + z[i] - 1 > r:
            l, r = i, i + z[i] - 1
    return z

print(*zfuncOpt(s))
print(*range(len(s)))

0 0 0 7 0 0 4 0 0 1
0 1 2 3 4 5 6 7 8 9


# Longest Palindromic Substring - Manacher's Alog *O(n)*

[YouTube video Link](https://www.youtube.com/watch?v=IvakBbsAhUU) good explanation.

Insight:  
suppose this is the string ***abcdedcba*** its palindrome and center is ***e***, now if we are right ***c*** than we can get the   
stored length till this is already a plaindrome from left ***c*** because they are inside a bigger palindrome and   
will have mirror symmetry. In this case it will be `0` length.  
for this string ***qabaeabap*** it will be `1` length at left ***b***.  

**NOTE:** In Manacher's algo, we exploit the fact the, if we already know a palindrome,  
then we can extract minimum distance till which the right side will be palindrome since we have already covered  
that exact string on the left side of the center of palindrome.

In [1]:
s = "babad"
s = "aybabtu"

# joining srtring with # to cover even length string cases
s = "#" + "#".join(s) + "#"
n = len(s)

center = 0
mlen, mindex = 0, 0
d = [0]*n
right = 0

for i in range(n):
    # use the mirror element to get the min length
    mirror = 2*center - i
    if right > i:
        d[i] = min(d[mirror], right-i)

    # these are left and right pointer to get palindrome, as we do in nomral O(n*n) algo
    a = i + (d[i]+1)
    b = i - (d[i]+1)

    while b>-1 and a<n and s[b] == s[a]:
        a+=1
        b-=1
        d[i]+=1

    # update the max values
    if d[i]>mlen:
        mlen = d[i]
        mindex = i

    # if we got new long palindrome udate the values and center
    if i+d[i]>right:
        center = i
        right = i + d[i]
    
ans = s[mindex-mlen:mindex+mlen+1].replace("#", "")
print(s)
print(ans)

#a#y#b#a#b#t#u#
bab


# good problem on trie

In [7]:
# problem link
# https://www.geeksforgeeks.org/googles-online-challenge-for-2021-intern-india-experience/?ref=rp
class node:
    def __init__(self, val=""):
        self.val = val
        self.end = 0
        self.d = dict()

def fill(root, w):
    if w == "": root.end = 1
    else:
        if w[0] not in root.d: root.d[w[0]] = node(w[0])
        fill(root.d[w[0]], w[1:])

lis = ['cat','map', 'bat','man','pen']
n = len(lis)
root = node()
for i in lis: fill(root, i)

def find(root, w):
    if w =="": return 1
    if w[0] == "?":
        ans = 0
        for i in root.d.values(): ans += find(i, w[1:])
        return ans
    elif w[0] not in root.d: return 0
    return find(root.d[w[0]], w[1:])
q = ["?at","ma?","?a?","??n"]
for a in q: print(find(root, a))

2
2
4
2


# Algorithms

## Counting inversions

### Using MergeSort

Do usual merge sort. But side by side keep counting inversions.  
In two sorted array number of inversions are like this.  
$ a[]$ and $b[]$ are subarray So actual array should be a+b.  
so all elements of $a[]$ should be less than equal to $b[]$.  
if not so then number of inversions are length of $a[]$ left from curr.  
now inversion if a+b are inversions in $a[]$ + $b[]$ + inversion during merging.

In [2]:
def merge(a, b):
    i = j = ans = 0
    n, m = len(a), len(b)
    c = []
    while i<n and j<m:
        if a[i]<=b[j]: c.append(a[i]); i+=1
        else: c.append(b[j]); j+=1; ans+=(n-i)
    while i<n: c.append(a[i]); i+=1
    while j<m: c.append(b[j]); j+=1
    return c, ans
def mergeSort(lis):
    p = len(lis)
    if p in {0, 1}: return lis, 0
    mid = p//2
    a, v1 = mergeSort(lis[:mid])
    b, v2 = mergeSort(lis[mid:])
    c, v3 = merge(a, b)
    return c, v1+v2+v3

lis = [1,2,6,7,5,4,3]
_, ans = mergeSort(lis)
print(ans)

9


### Using Fenwick Tree
Make a **BIT** (binary index tree) array. Then from reverse keep adding 1 for that element's mapped postition in **BIT** (here its the same becuase it permuation array but not always this happens).  
Before adding 1 in bit, add sum upto element-1(not equal to elements) to answer.

In [3]:
lis = [1,2,6,7,5,4,3]
n = len(lis)
bit = [0]*(n+1)

def update(pos, delta):
    pos += 1
    while pos<n+1:
        bit[pos] += delta
        pos += (-pos & pos)

def presum(pos):
    ans = 0
    pos += 1
    while pos>0:
        ans += bit[pos]
        pos -= (-pos & pos)
    return ans

ans = 0
for i in range(n-1, -1, -1):
    ans += presum(lis[i]-2)
    update(lis[i]-1, 1)
print(ans)

9


here is an example of a elegent mapping.

In [4]:
from bisect import *

# finding inversions
lis = [1,2,3,4,5,6,0]
n = len(lis)
l = n+1
bit = [0]*l
def update(pos, delta):
    pos += 1
    while pos<l:
        bit[pos] += delta
        pos += (-pos & pos)
def count(pos):
    pos += 1
    ans = 0
    while pos>0:
        ans += bit[pos]
        pos -= (-pos & pos)
    return ans

sor = sorted(set(lis)) # d = dict((val, i) for i, val in enumerate(sorted(set)))
ans = 0
# so here basically you mapped elements to index of lis by sorting it, and you keep updating index position by finding postion of it using binary
# search.
for i in reversed(lis):
    ans += count(bisect_left(sor, i)) # here it could be optimized by using map
    update(bisect_left(sor, i), 1) # here it could be optimized by using map
print(ans)

6


## Closest Pair in a Plane

In [5]:
# lis = [[x, y, index]... ]
lis = [[0, 0, 0], [-4, 1, 1], [-7, -2, 2], [4, 5, 3], [1, 1, 4]]
n = len(lis)
lis.sort() # sorting wrt x

def dis(a, b):
    return ((a[0] - b[0])**2 + (a[1]  - b[1])**2)**0.5

def brt(p, y=None):
    from bisect import bisect_left, bisect
    n = len(p)
    ans = 1e10
    a, b = -1, -1
    if y is None:
        for i in range(n):
            for j in range(i+1, n):
                val = dis(p[i], p[j])
                if val<ans:
                    ans = val
                    a, b = min(p[i][2], p[j][2]), max(p[i][2], p[j][2])
        return ans, a, b
    p.sort(key=lambda x: x[1])
    for i in range(n):
        j = i-1
        while j>=0 and p[i][1]-p[j][1]<=y:
            val = dis(p[i], p[j])
            if val<ans:
                ans = val
                a, b = min(p[i][2], p[j][2]), max(p[i][2], p[j][2])
            j-=1   
        j = i+1
        while j<n and p[j][1]-p[i][1]<=y:
            val = dis(p[i], p[j])
            if val<ans:
                ans = val
                a, b = min(p[i][2], p[j][2]), max(p[i][2], p[j][2])
            j+=1
        return ans, a, b

def sol(p):
    n = len(p)
    if n<=2: return brt(p)
    mid = n//2
    d1, a1, b1 = sol(p[:mid])
    d2, a2, b2 = sol(p[mid:])
    d, a, b = (d1, a1, b1) if d1<d2 else (d2, a2, b2)
    
    # so getting all points in 
    s = []
    linex = p[mid][0]
    for i in range(mid-1, -1, -1):
        if linex-p[i][0]<d: s.append(p[i])
        else: break
    for i in range(mid, n):
        if p[i][0]-linex<d: s.append(p[i])
        else: break
    d0, a0, b0 = brt(s, d)
    d, a, b = (d0, a0, b0) if d0<d else (d, a, b)
    return d, a, b

d, a, b = sol(lis)
print(a, b,'{:.6f}'.format(d))

0 4 1.414214


## Overlapping intervals - easy solution

In [6]:
intervals = [(1, 3), (2, 4), (6, 8), (9, 10)]
intervals.sort()
ans = []
for a, b in intervals:
    if ans and a<=ans[-1][1]: 
        ans[-1][1] = max(ans[-1][1], b)
    else:
        ans.append([a, b])
print(ans)

[[1, 4], [6, 8], [9, 10]]


## Farthest Right Maximum

Idea is to sort the array, now for a postions $x$ the ones in right are the only ones greater than it so get the max index out of them, which can be easily done by right passing the array.

In [7]:
A = [10, 4, 5, 7, 1]
n = len(A)
lis = [(A[i], i) for i in range(n)]
lis.sort()
print(lis)
right = [0]*n
m = 0
for i in range(n-1, -1, -1):
    right[i] = m = max(m, lis[i][1])
ans = [0]*n
for i in range(n):
    ans[lis[i][1]] = right[i]
print(ans)

[(1, 4), (4, 1), (5, 2), (7, 3), (10, 0)]
[0, 3, 3, 3, 4]


## Integer to Roman

So generally if you would have to do this for base 15 or any other you would have made check points for every multiple of 15 by dividing number by 15 every time and passed remainder.  
~~~~
number = 145
base = 15
ans = []
while number:
    ans.append(number//base) # creating check point
    number = number % base # passing remainder
print(ans) # in base 15 with usual sysmbols
~~~~
`NOTE: python is awesome, to convert any number to another base just do this`  
`ans = int(str(number), base)`  
but here check points are a bit weird and so you have to manually make a list of them and a map for sysmbols.  
Like for digits **4** and and **9**.

In [8]:
#code
def roman(A):
    values = [ 1000, 900, 500, 400, 100, 90, 50, 40, 10, 9, 5, 4, 1 ] # check points
    numerals = [ "M", "CM", "D", "CD", "C", "XC", "L", "XL", "X", "IX", "V", "IV", "I" ] # map
    res = ""
    for i, v in zip(numerals, values):
        res += (A//v) * i
        A %= v
    return res
roman(3999)

'MMMCMXCIX'

In [9]:
# Roman numerals to Integer

def numeral(s):
    n = len(s)
    i = 0
    ans = 0
    c = ["M", "D", "C", "L", "X", "V", "I"]
    v = [1000, 500, 100, 50, 10, 5, 1]

    while i<n:
        e = i
        while i+1<n and s[i+1] == s[i]: i+=1
        l = i-e+1

        if l == 1:
            if i+1<n and c.index(s[i])>c.index(s[i+1]):
                ans += v[c.index(s[i+1])] - v[c.index(s[i])]
                i+=1
            else: ans += v[c.index(s[i])]
        else: ans += l*(v[c.index(s[i])])
        i+=1
    return ans

for i in range(4): 
    print(numeral(roman(i)))

0
1
2
3


## Two pointer nice solution

Question: Given a sorted array, remove the duplicates in place such that each element can appear atmost twice and return the new length.  
 `A = [1,1,1,2]`
 `converted = [1,1,2].`

In [1]:
# code 
A = [1,1,1,2,2,2,3,3,3,4,4,4,4,5,5,5,5,6,6,6]
# A = [1,2,3,4,5]
def sol(A):
    n = len(A)
    if n <= 1: return n
    index = 0
    for i in range(n):
        if index<2 or A[i] != A[index-2]:
            A[index] = A[i]
            index+=1
    for i in range(n-index): A.pop()
    return len(A)
sol(A)
print(A)

[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6]


## All permutations 
To build permutations of length n, we need permutation of (n-1) length.  
The below code build therefore builds the entire premutation tree of all possible length.  
At level 0, we have original list.  
At level 1 we build permutation of length 1, by swaping 1st index by all possible value from list.  
Then at level 2 we build all possible permutations for all branches by swapping 2nd index for all   
possible values except first index value of that branch.  
This keeps going on.  

<!-- ![image](https://media.geeksforgeeks.org/wp-content/cdn-uploads/NewPermutation.gif) -->
<img src="https://media.geeksforgeeks.org/wp-content/cdn-uploads/NewPermutation.gif" alt="drawing" style="width:700px;"/>


In [11]:
arr = [1,2,3]
k = 2
# swapping methond
def allPermuationInpalce(a, l):
    print(a[:l]) # use condition to get a specific level permuation
    for i in range(l, len(a)):
        a[i], a[l] = a[l], a[i]
        allPermuationInpalce(a, l+1)
        a[i], a[l] = a[l], a[i]

# list reduction method
def allPermuation(a, remain):
    print(a) # use condition to get a specific level permuation
    for i in range(len(remain)):
        allPermuation(a + [remain[i]], remain[:i] + remain[i+1:])

# allPermuation([], arr[:])
allPermuationInpalce(arr, 0)

[]
[1]
[1, 2]
[1, 2, 3]
[1, 3]
[1, 3, 2]
[2]
[2, 1]
[2, 1, 3]
[2, 3]
[2, 3, 1]
[3]
[3, 2]
[3, 2, 1]
[3, 1]
[3, 1, 2]


## All SubSets

simple $2^n$ concept, either select the element or not.

In [12]:
lis = [1,2,3]
n = len(lis)

def allSubSet(i=0, a=[]):
    if i == n:
        print(a)
        return

    allSubSet(i+1, a)
    allSubSet(i+1, a + [lis[i]])

allSubSet()

[]
[3]
[2]
[2, 3]
[1]
[1, 3]
[1, 2]
[1, 2, 3]


## All possible Combination of K

In [13]:
lis = [1,2,3,4]
n = len(lis)
k = 3

def allCombination(i=0, a=[]):
    if len(a) == k:
        print(a)
        return
    if i == n:
        return
    allCombination(i+1, a)
    allCombination(i+1, a+[lis[i]])
    
allCombination()

[2, 3, 4]
[1, 3, 4]
[1, 2, 4]
[1, 2, 3]


## Next Permuations

In [16]:
# NOTE: after finding index do remember that elements after index are reverse sorted, don't remember why? Think then!
A = [1,2,3,4,6,5]
n = len(A)
index = 0
# getting last unscrambled index
for i in range(n-1):
    if A[i]<A[i+1]:
        index = i

# means reverse sorted so return sorted one
if index == -1:
    print(sorted(A))
else:
    # get last element just bigger than index element
    for i in range(n-1, index, -1):
        if A[i]>A[index]:
            A[index], A[i] = A[i], A[index]
            break
    # now reverse the elements after index elements
    A = A[:index+1] + A[index+1:][::-1]
    print(A)

[1, 2, 3, 5, 4, 6]


## Median finding Algorithm

### Quick Select

In [14]:
from random import choice

def QS(lis, k):
    if len(lis)<=5: return sorted(lis)[k]
    pivot = choice(lis)
    low = [i for i in lis if i<pivot]
    equal = [i for i in lis if i == pivot]
    high = [i for i in lis if i>pivot]
    l, e = len(low), len(equal)
    
    if k<l: return QS(low, k)
    elif k>=l+e: return QS(high, k-l-e)
    else: return pivot
    
QS([1,2,3,4,5,6,7,8,9,0], 3)

3

### Median Of Medians
the only problem with quick Select is that its random, therefore we can have $O(N^2)$ time complexity as in quick sort.  
In MOM we get median by clever way of sorting medians.

1. subdividing into set of 5.
2. collecting medians of those small sets(simple sort).
3. finding pivot, if around 5 then simple sort, else use recursion.
4. quick sorting technique to partition around pivot.
5. compare the rank(position) of pivot to what rank you want to find.

In [18]:
def MOM(lis, k):
    if len(lis)<=5: return sorted(lis)[k]
    medians = [sorted(lis[i:i+5])[min(2, (len(lis)-i))//2] for i in range(0, len(lis), 5)]
    pivot = MOM(medians, len(medians)//2)
    low = [i for i in lis if i<pivot]
    equal = [i for i in lis if i == pivot]
    high = [i for i in lis if i>pivot]
    r = len(low)
    p = len(low) + len(equal)
    if k<r: return MOM(low, k)
    elif k>=r+p: return MOM(high, k-r-p)
    else: return pivot
    
lis = [5,5,5,5,5,5,5,5,5]
MOM(lis, 3)

5

### Median of 2 sorted lists

In [19]:
import random
def solve(a, b):
    def kth(a, b, k):
        if len(a) > len(b): return kth(b, a, k)
        if len(a) == 0: return b[k-1]
        if k == 1: return min(a[0], b[0])

        i = min(len(a), k//2)
        j = k - i

        if a[i-1] <= b[j-1]: return kth(a[i:], b[:j], k-i)
        else: return kth(a[:i], b[j:], k-j)

    n, m = len(a), len(b)
    k1 = 1 + (n+m)//2   # right mid
    k2 = 1 + (n+m-1)//2 # left mid

    return 0.5*(kth(a, b, k1) + kth(a, b, k2))

a = sorted(random.sample(range(0,99), random.randint(0,10)))
b = sorted(random.sample(range(0,99), random.randint(0,10)))
solve(a, b)

51.0

# Mo's Algorithm

Very important [link](https://www.hackerearth.com/practice/notes/mos-algorithm/) to refer.
  
So basic idea is to sort the querry in intervals such that you don't have to move mo's right and mo's left not more than $n\sqrt{n}$.
And rule for that is.  
Rearrange all queries in a way we will call “Mo’s order”. It is defined like this: `[L1, R1]` comes earlier than `[L2, R2]` in Mo’s order if and only   if:  
   a) $\frac{L1}{BLOCK\_SIZE} < \frac{L2}{BLOCK\_SIZE}$  
   b) $\frac{L1}{BLOCK\_SIZE} = \frac{L2}{BLOCK\_SIZE}$ and $R1 < R2$  

[Here](https://codeforces.com/problemset/problem/86/D) is the question and below is the solution code.  

In [ ]:
import sys; #input = sys.stdin.readline

n, q = map(int, input().split())
lis = list(map(int, input().split()))
querry = []
for i in range(q): 
    a,b = map(int, input().split())
    querry.append((i, a, b))

D = int(n**0.5)
querry.sort(key=lambda x: (x[1]/D, x[2])) # sorting by the criteria
cnt = [0]*int(1e6)
ans = [0]*q
ml, mr = 0, -1
cur = 0
for i, l, r in querry:
    l-=1; r-=1
    while ml<l:
        p = cnt[lis[ml]]
        cnt[lis[ml]] -= 1
        # cur += (cnt[lis[ml]]**2 - p**2)*lis[ml]
        cur -= (2*p -1)*lis[ml]
        ml+=1
    while ml>l:
        ml-=1
        p = cnt[lis[ml]]
        cnt[lis[ml]] += 1
        # cur += (cnt[lis[ml]]**2 - p**2)*lis[ml]
        cur += (2*p+1)*lis[ml]
    while mr<r:
        mr+=1
        p = cnt[lis[mr]]
        cnt[lis[mr]] += 1
        # cur += (cnt[lis[mr]]**2 - p**2)*lis[mr]
        cur += (2*p+1)*lis[mr]
    while mr>r:
        p = cnt[lis[mr]]
        cnt[lis[mr]] -= 1
        # cur += (cnt[lis[ml]]**2 - p**2)*lis[ml]
        cur -= (2*p -1)*lis[mr]
        mr-=1
    ans[i] = cur
for i in ans: print(i)
